In [1]:
# Import required libraries
import sys
import pandas as pd
from pathlib import Path

# Add src directory to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.database.connection import DatabaseConnection
from src.database.schema import get_database_stats

print("📊 Health Data Analytics System")
print("=" * 40)


📊 Health Data Analytics System


In [2]:
# Connect to database
db_path = project_root / "data" / "health_data.db"
print(f"🔗 Connecting to database: {db_path}")

if not db_path.exists():
    print("❌ Database not found. Please run setup first:")
    print("   python scripts/setup_new_database.py")
else:
    db_conn = DatabaseConnection(str(db_path))
    print("✅ Connected successfully!")


🔗 Connecting to database: /Users/mgartner/Documents/personal_projects/healthdatabase/data/health_data.db
✅ Connected successfully!


In [3]:
# Get database statistics
stats = get_database_stats(db_conn)

print("\n📈 Database Statistics")
print("=" * 25)
print(f"Schema Version: {stats.get('schema_version', 'unknown')}")
print(f"Total Records: {stats.get('total_records', 0):,}")
print(f"Total Tables: {len(stats.get('tables', {}))}")

print("\n📋 Table Overview")
print("=" * 20)
for table_name, table_stats in stats.get('tables', {}).items():
    print(f"{table_name}: {table_stats['row_count']:,} records")
    if 'date_range' in table_stats:
        date_range = table_stats['date_range']
        print(f"   📅 {date_range['min_date']} to {date_range['max_date']}")



📈 Database Statistics
Schema Version: 2.0
Total Records: 1,001
Total Tables: 4

📋 Table Overview
users: 1 records
daily_activity: 500 records
   📅 2023-01-04 to 2024-09-28
sleep_data: 500 records
   📅 2023-01-04 to 2024-09-28
heart_rate_data: 0 records


In [4]:
# Load users data
users_query = "SELECT * FROM users"
users_result = db_conn.execute_query(users_query)
users_df = pd.DataFrame(users_result)

print(f"📊 Users DataFrame: {len(users_df)} records")
print("\n🔍 Data Preview:")
users_df.head()


📊 Users DataFrame: 1 records

🔍 Data Preview:


,0,1,2,3,4,5,6
0,1,default,Default User,None,UTC,2025-06-04 16:05:24,2025-06-04 16:05:24


In [6]:
# Load activity data
activity_query = "SELECT * FROM daily_activity ORDER BY date"
activity_result = db_conn.execute_query(activity_query)
activity_df = pd.DataFrame(activity_result)
activity_df


,0,1,2,3,4,5,6,7,8,9,10
0,1,1,2023-01-04,5351,202.0,4160.0,0.0,0,zepp,2025-06-04 16:05:37,2025-06-04 16:05:37
1,2,1,2023-01-05,8855,326.0,6929.0,0.0,0,zepp,2025-06-04 16:05:37,2025-06-04 16:05:37
2,3,1,2023-01-06,19256,770.0,16358.0,0.0,0,zepp,2025-06-04 16:05:37,2025-06-04 16:05:37
3,4,1,2023-01-07,13662,520.0,10902.0,0.0,0,zepp,2025-06-04 16:05:37,2025-06-04 16:05:37
4,5,1,2023-01-08,14656,557.0,12268.0,0.0,0,zepp,2025-06-04 16:05:37,2025-06-04 16:05:37
...,...,...,...,...,...,...,...,...,...,...,...
495,496,1,2024-09-24,26317,954.0,21050.0,8498.0,0,zepp,2025-06-04 16:05:37,2025-06-04 16:05:37
496,497,1,2024-09-25,24815,893.0,20024.0,8337.0,0,zepp,2025-06-04 16:05:37,2025-06-04 16:05:37
497,498,1,2024-09-26,23833,903.0,18764.0,3792.0,0,zepp,2025-06-04 16:05:37,2025-06-04 16:05:37
498,499,1,2024-09-27,14622,568.0,11329.0,344.0,0,zepp,2025-06-04 16:05:37,2025-06-04 16:05:37


In [7]:
# Load sleep data
sleep_query = "SELECT * FROM sleep_data ORDER BY date"
sleep_result = db_conn.execute_query(sleep_query)
sleep_df = pd.DataFrame(sleep_result)

# Convert date and timestamp columns for better analysis
if not sleep_df.empty:
    sleep_df['date'] = pd.to_datetime(sleep_df['date'])
    if 'sleep_start' in sleep_df.columns:
        sleep_df['sleep_start'] = pd.to_datetime(sleep_df['sleep_start'])
    if 'sleep_end' in sleep_df.columns:
        sleep_df['sleep_end'] = pd.to_datetime(sleep_df['sleep_end'])

print(f"📊 Sleep DataFrame: {len(sleep_df)} records")
print("\n🔍 Data Preview:")
sleep_df.head()


KeyError: 'date'

In [ ]:
# Load heart rate data (currently empty but schema exists)
hr_query = "SELECT * FROM heart_rate_data ORDER BY timestamp"
hr_result = db_conn.execute_query(hr_query)
hr_df = pd.DataFrame(hr_result)

# Convert timestamp column if data exists
if not hr_df.empty:
    hr_df['timestamp'] = pd.to_datetime(hr_df['timestamp'])

print(f"📊 Heart Rate DataFrame: {len(hr_df)} records")
if not hr_df.empty:
    print("\n🔍 Data Preview:")
    print(hr_df.head())
else:
    print("💡 No heart rate data yet. Schema is ready for future imports!")
    print("\n📋 Available columns:")
    # Show the schema structure
    schema_query = "PRAGMA table_info(heart_rate_data)"
    schema_result = db_conn.execute_query(schema_query)
    schema_df = pd.DataFrame(schema_result)
    print(schema_df[['name', 'type']].to_string(index=False))


In [ ]:
# Summary of all data sources
print("🏥 HEALTH DATA SUMMARY")
print("=" * 50)

# Activity Data Summary
if not activity_df.empty:
    print(f"\n🏃 ACTIVITY DATA ({len(activity_df)} records)")
    print(f"   📅 Date Range: {activity_df['date'].min().date()} to {activity_df['date'].max().date()}")
    print(f"   👣 Average Steps: {activity_df['steps'].mean():.0f}")
    print(f"   🔥 Average Calories: {activity_df['calories'].mean():.0f}")
    print(f"   📏 Average Distance: {activity_df['distance'].mean():.2f} km")

# Sleep Data Summary
if not sleep_df.empty:
    # Filter records with actual sleep data
    sleep_with_data = sleep_df[sleep_df['total_sleep_minutes'] > 0]
    print(f"\n😴 SLEEP DATA ({len(sleep_with_data)} records with sleep)")
    if not sleep_with_data.empty:
        print(f"   📅 Date Range: {sleep_with_data['date'].min().date()} to {sleep_with_data['date'].max().date()}")
        print(f"   ⏰ Average Sleep: {sleep_with_data['total_sleep_minutes'].mean():.0f} minutes ({sleep_with_data['total_sleep_minutes'].mean()/60:.1f} hours)")
        print(f"   🌊 Average Deep Sleep: {sleep_with_data['deep_sleep_minutes'].mean():.0f} minutes")
        print(f"   🌙 Average Light Sleep: {sleep_with_data['light_sleep_minutes'].mean():.0f} minutes")
        print(f"   🧠 Average REM Sleep: {sleep_with_data['rem_sleep_minutes'].mean():.0f} minutes")
        print(f"   ⚡ Average Sleep Efficiency: {sleep_with_data['sleep_efficiency'].mean():.1f}%")

# Heart Rate Data Summary
print(f"\n❤️ HEART RATE DATA ({len(hr_df)} records)")
if hr_df.empty:
    print("   💡 Schema ready for future heart rate data imports")

print(f"\n👥 USERS: {len(users_df)} registered")

print("\n✅ Data overview complete! All datasets loaded successfully.")
